# Data Processing

This notebook demonstrates how to use `tgml` for common data processing tasks on graphs stored in `TigerGraph`.

## Define Graph

Conceptually, the `TigerGraph` class represents the graph stored in the database. Under the hood, it stores the necessary information to communicate with the TigerGraph database. It can read `username` and `password` from environment variables `TGUSERNAME` and `TGPASSWORD`. Hence, we recommend storing those credentials in the environment variables or in a `.env` file instead of hardcoding them in code. However, if you do provide `username` and `password` to this class constructor, the environment variables will be ignored.

In [ ]:
from tgml.data import TigerGraph

Args to the `TigerGraph` class:
*    host (str, ): Address of the server. Defaults to "http://localhost".
*    graph (str, ): Name of the graph. Defaults to None.
*    username (str, optional): Username. Defaults to None.
*    password (str, optional): Password for the user. Defaults to None.
*    rest_port (str, optional): Port for the REST endpoint. Defaults to "9000".
*    gs_port (str, optional): Port for GraphStudio. Defaults to "14240".
*    token_auth (bool, optional): Whether to use token authentication. If True, token authentication must be turned on in the TigerGraph database server. Defaults to True.

In [ ]:
host = "http://protomolecule.magichome"
graphName = "Cora"
username = "tigergraph"
password = "Tigergraph"
restppPort = 9000       # default 9000
gsPort     = 14240       # default 14240
mysecret = "enigs3sm45lrhkm6j5jt914bk586u3sn"
tgraph = TigerGraph("http://protomolecule.magichome", "Cora", "tigergraph", "Tigergraph", token_auth=False)

In [ ]:
host = "http://protomolecule.magichome"
graphName = "Cora"
username = "tigergraph"
password = "Tigergraph"
restppPort = 9000       # default 9000
gsPort     = 14240       # default 14240
mysecret = "enigs3sm45lrhkm6j5jt914bk586u3sn"

# host = "http://35.230.92.92"
# password = "tigergraphml"

# First establish a basic connection using a secret.  Do *not* do this if you already have a token
# import pyTigerGraph as tg
# conn = tg.TigerGraphConnection(host=host, restppPort=restppPort, gsPort=gsPort, graphname=graphName, password=password)
# token = conn.getToken(mysecret, setToken=True, lifetime=None)[0]

tgraph = TigerGraph("http://protomolecule.magichome", "Cora", "tigergraph", "Tigergraph" )
# tgraph = TigerGraph(host = host,
#                     graph = graphName,
#                     username = username,
#                     password = password)

In [ ]:
tgraph.info()

In [ ]:
tgraph.number_of_vertices()

In [ ]:
tgraph.number_of_vertices("Paper")

In [ ]:
tgraph.number_of_edges()

In [ ]:
tgraph.number_of_edges("Cite")

## Train/Validation/Test Split

In [ ]:
from tgml.utils import split_vertices

`tgml` provide a utility function `split_vertices` to split vertices into a training, a validation, and a test set. More precisely, it creates 3 boolean attributes with each attribute indicating whether the vertex is in the corresponding set. For example, if you want to split the vertices into 80% train, 10% validation and 10% test, you can provide as arguments to the function `train_mask=0.8, val_mask=0.1, test_mask=0.1`. This will create 3 attributes `train_mask`, `val_mask`, `test_mask` in the graph, if they don't already exist. 80% of vertices will be set to `train_mask=1`, 10% to `val_mask=1`, and 10% to `test_mask=1` at random. There will be no overlap between the partitions. You can name the attributes however you like as long as you follow the format, such as `yesterday=0.8, today=0.1, tomorrow=0.1`, but we recommend something  meaningful. 

In [ ]:
split_vertices(tgraph, train_mask=0.8, val_mask=0.1, test_mask=0.1)

Now the split is done. Load all vertices and check if the split is correct. See the next tutorial for details on `VertexLoader` and other data loaders.

In [ ]:
from tgml.dataloaders import VertexLoader

In [ ]:
%%time
vertex_loader = VertexLoader(tgraph, attributes="train_mask,val_mask,test_mask")

In [ ]:
%%time
data = vertex_loader.data

In [ ]:
data.train_mask.sum()/len(data), data.val_mask.sum()/len(data), data.test_mask.sum()/len(data)

## Feature Engineering

We are adding graph algorithms to the workbench to perform feature engineering tasks. Current we are experimenting with PageRank and more details will be added shortly. For now you can use `pyTigerGraph` to install and run your own GSQL queries for feature extraction or any other task that can be done in GSQL.

In [ ]:
import pyTigerGraph as tg

conn = tg.TigerGraphConnection(
    host="http://35.230.92.92",
    username="tigergraph",
    password="tigergraphml",
    graphname="Cora",
)

conn.apiToken = conn.getToken(conn.createSecret())

### PageRank

PageRank is the algorithm that originally ran Google's search engine, where it ranked the most influential webpages higher than less influential ones. A page was determined to be influential through its PageRank score, which is based off the importance of the pages that linked to it. Generalizing this, PageRank finds the most influential vertices in a graph based upon how influential the vertices that have edges to the desired vertex are.

The documentation of the PageRank query is found here: https://docs.tigergraph.com/graph-algorithm-library/centrality/pagerank.

#### Install PageRank Query

In [ ]:
gsql = '''
CREATE QUERY tg_pagerank (STRING v_type, STRING e_type,
 FLOAT max_change=0.001, INT max_iter=25, FLOAT damping=0.85, INT top_k = 100,
 BOOL print_accum = TRUE, STRING result_attr =  "", STRING file_path = "",
 BOOL display_edges = FALSE) {
/*
 Compute the pageRank score for each vertex in the GRAPH
 In each iteration, compute a score for each vertex:
     score = (1-damping) + damping*sum(received scores FROM its neighbors).
 The pageRank algorithm stops when either of the following is true:
 a) it reaches max_iter iterations;
 b) the max score change for any vertex compared to the last iteration <= max_change.
 v_type: vertex types to traverse          print_accum: print JSON output
 e_type: edge types to traverse            result_attr: INT attr to store results to
 max_iter; max #iterations                 file_path: file to write CSV output to
 top_k: #top scores to output              display_edges: output edges for visualization
 max_change: max allowed change between iterations to achieve convergence
 damping: importance of traversal vs. random teleport

 This query supports only taking in a single edge for the time being (8/13/2020).
*/
	TYPEDEF TUPLE<VERTEX Vertex_ID, FLOAT score> Vertex_Score;
	HeapAccum<Vertex_Score>(top_k, score DESC) @@topScores;
	MaxAccum<FLOAT> @@max_diff = 9999;    # max score change in an iteration
	SumAccum<FLOAT> @recvd_score = 0; # sum of scores each vertex receives FROM neighbors
	SumAccum<FLOAT> @score = 1;           # initial score for every vertex is 1.
	SetAccum<EDGE> @@edgeSet;             # list of all edges, if display is needed
	FILE f (file_path);

# PageRank iterations	
	Start = {v_type};                     # Start with all vertices of specified type(s)
	WHILE @@max_diff > max_change LIMIT max_iter DO
			@@max_diff = 0;
			V = SELECT s
				FROM Start:s -(e_type:e)-> v_type:t
				ACCUM t.@recvd_score += s.@score/(s.outdegree(e_type)) 
				POST-ACCUM s.@score = (1.0-damping) + damping * s.@recvd_score,
						   s.@recvd_score = 0,
						   @@max_diff += abs(s.@score - s.@score');
	END; # END WHILE loop

# Output
	IF file_path != "" THEN
	  f.println("Vertex_ID", "PageRank");
	END;

	V = SELECT s FROM Start:s
		POST-ACCUM 
			IF result_attr != "" THEN s.setAttr(result_attr, s.@score) END,
			IF file_path != "" THEN f.println(s, s.@score) END,
			IF print_accum THEN @@topScores += Vertex_Score(s, s.@score) END;

	IF print_accum THEN
		PRINT @@topScores;
		IF display_edges THEN
			PRINT Start[Start.@score];
			Start = SELECT s
					FROM Start:s -(e_type:e)-> v_type:t
					ACCUM @@edgeSet += e;
		   PRINT @@edgeSet;
		END;
	END;
}
'''

In [ ]:
# You can write any gsql query and install it with the following command.
print(conn.gsql(gsql + "\n INSTALL QUERY tg_pagerank"))

#### Run PageRank Query

In [ ]:
# Once your query is installed, you can run it using the 
# `runInstalledQuery` function with input parameters as a dict.

params = {
  "v_type": "Paper",
  "e_type": "Cite",
  "max_change": 0.001,
  "max_iter": 25,
  "damping": 0.85,
  "top_k": 150,
  "print_accum": True,
  "result_attr": "",
  "file_path": "",
  "display_edges": False
}

pr_res = conn.runInstalledQuery("tg_pagerank", params=params)

In [ ]:
# Output from your query will be returned as a JSON
pr_res